# Group 2 Final Project Technical Notebook

Name(s): Dominic Fanucchi, Bilal Najar, Carlo Sanchez  
Date:    September 29, 2023

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
import math
import scipy.stats as stats
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KernelDensity
import seaborn as sns
import random
from scipy.stats import norm, dgamma, geom, t

In [2]:
#pip install ucimlrepo
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
mushroom = fetch_ucirepo(id=73) 
  
# data (as pandas dataframes) 
X = mushroom.data.features 
y = mushroom.data.targets 
  
# metadata 
#print(mushroom.metadata) 
  
# variable information 
#print(mushroom.variables) 

In [3]:
#set the data into a single dataframe
mushroom_df = X.join(y, lsuffix='X', rsuffix='y')
#mushroom_df.describe()
#print(mushroom_df.head())

#get columns of the dataset
columns = list(mushroom_df.columns)
#print(columns)

#examining possible missing values
#print("Missing values distribution: ")
#print(mushroom_df.isnull().mean())
mushroom_df=mushroom_df.dropna()
print("")
#there is missing data from the stalk-root column
print(mushroom_df.isnull().mean())

#print(mushroom_df)

print(mushroom_df['stalk-root'])
print(mushroom_df)


cap-shape                   0.0
cap-surface                 0.0
cap-color                   0.0
bruises                     0.0
odor                        0.0
gill-attachment             0.0
gill-spacing                0.0
gill-size                   0.0
gill-color                  0.0
stalk-shape                 0.0
stalk-root                  0.0
stalk-surface-above-ring    0.0
stalk-surface-below-ring    0.0
stalk-color-above-ring      0.0
stalk-color-below-ring      0.0
veil-type                   0.0
veil-color                  0.0
ring-number                 0.0
ring-type                   0.0
spore-print-color           0.0
population                  0.0
habitat                     0.0
poisonous                   0.0
dtype: float64
0       e
1       c
2       c
3       e
4       e
       ..
7986    b
8001    b
8038    b
8095    c
8114    c
Name: stalk-root, Length: 5644, dtype: object
     cap-shape cap-surface cap-color bruises odor gill-attachment  \
0            x         